In [1]:
from dotenv import load_dotenv
import requests
import os
import pandas as pd
import overpy

In [3]:
api = overpy.Overpass()

In [51]:
query = """
[out:json];
area[name="New York"]->.searchArea;
(
  node["amenity"="pub"](area.searchArea);
  node["amenity"="bar"](area.searchArea);
  node["amenity"="biergarten"](area.searchArea);
  node["amenity"="cafe"](area.searchArea);
  node["amenity"="fast_food"](area.searchArea);
  node["amenity"="food_court"](area.searchArea);
  node["amenity"="ice_cream"](area.searchArea);
  node["amenity"="pub"](area.searchArea);
  node["amenity"="restaurant"](area.searchArea);
);
out center;
"""

In [52]:
result = api.query(query)

In [53]:
for p in dir(result.nodes[0]):
    print(p)

__annotations__
__class__
__delattr__
__dict__
__dir__
__doc__
__eq__
__format__
__ge__
__getattribute__
__getstate__
__gt__
__hash__
__init__
__init_subclass__
__le__
__lt__
__module__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__sizeof__
__str__
__subclasshook__
__weakref__
_result
_type_value
attributes
from_json
from_xml
get_center_from_json
get_center_from_xml_dom
id
lat
lon
tags


In [54]:
print(set(result.nodes[0].tags.keys()))

{'gnis:feature_id', 'amenity', 'name', 'ele'}


In [55]:
params = set()

for node in result.nodes:
    print(set(node.tags.keys()))
    params = params.union(set(node.tags.keys()))

print(params)

{'gnis:feature_id', 'amenity', 'name', 'ele'}
{'cuisine', 'name', 'addr:street', 'amenity', 'website', 'addr:housenumber', 'addr:postcode'}
{'amenity', 'name'}
{'cuisine', 'name', 'addr:street', 'amenity', 'website', 'addr:housenumber', 'addr:postcode'}
{'amenity', 'name'}
{'cuisine', 'addr:city', 'name', 'addr:state', 'addr:street', 'amenity', 'phone', 'website', 'addr:housenumber', 'addr:postcode'}
{'cuisine', 'addr:city', 'name', 'addr:state', 'addr:street', 'amenity', 'phone', 'website', 'addr:housenumber', 'addr:postcode'}
{'cuisine', 'addr:city', 'name', 'addr:state', 'addr:street', 'amenity', 'phone', 'website', 'addr:housenumber', 'addr:postcode'}
{'created_by', 'amenity', 'name'}
{'created_by', 'amenity', 'name'}
{'created_by', 'amenity', 'name'}
{'cuisine', 'addr:city', 'name', 'nysgissam:nysaddresspointid', 'addr:state', 'addr:street', 'amenity', 'addr:housenumber', 'addr:postcode'}
{'amenity', 'name'}
{'amenity', 'name'}
{'amenity', 'name'}
{'amenity', 'name'}
{'cuisine', '

In [56]:
print(len(result.nodes))

21607


In [57]:
cols = 

SyntaxError: invalid syntax (846146112.py, line 1)